In [ ]:
#import sep
import astropy.io.fits
import numpy as np
import matplotlib.pyplot as plt
from matplotlib import rcParams
from astropy.io import fits

In [ ]:
%matplotlib inline

rcParams['figure.figsize'] = [10., 4.]

In [ ]:
# read WHT image into standard 2-d numpy array and SCI image plot

uri1 = "https://s3.amazonaws.com/grizli-v2/JwstMosaics/v6/sub-primer-uds-grizli-v6.0-ir_drc_sci.fits"
uri2 = "https://s3.amazonaws.com/grizli-v2/JwstMosaics/v6/sub-primer-uds-grizli-v6.0-ir_drc_wht.fits"

with fits.open(uri1, use_fsspec=True, fsspec_kwargs={"anon": True}) as hdul:  
   data = hdul[0].data
   


In [ ]:
with fits.open(uri2, use_fsspec=True, fsspec_kwargs={"anon": True}) as hdul:  
   data1 = hdul[0].data

wht_data = np.array(data1.data)

hdu_list = fits.open(uri1)
hdu_list.info()
print(data.shape)
hdu_list.close()

hdu_list = fits.open(uri2)
hdu_list.info()
print(wht_data.shape)
print(type(wht_data))
hdu_list.close()

In [ ]:
# show the image
m, s = np.mean(data), np.std(data)
plt.imshow(data, interpolation='nearest', cmap='gray', vmin=m-s, vmax=m+s, origin='lower')
plt.colorbar();

In [ ]:
data = data.byteswap().newbyteorder()
# measure a spatially varying background on the image
#bkg1 = sep.Background(data1)
bkg = sep.Background(data, bw=64, bh=64, fw=3, fh=3)

# get a "global" mean and noise of the image background:
print("Global mean = ",bkg.globalback)
print("Global noise = ",bkg.globalrms)

# evaluate background as 2-d array, same size as original image
bkg_image = bkg.back()
# bkg_image = np.array(bkg) # equivalent to above

# show the background
plt.imshow(bkg_image, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# evaluate the background noise as 2-d array, same size as original image
bkg_rms = bkg.rms()

In [ ]:
# show the background noise
plt.imshow(bkg_rms, interpolation='nearest', cmap='gray', origin='lower')
plt.colorbar();

# subtract the background
data_sub = data - bkg
#plt.imshow(data_sub, interpolation='nearest', cmap='gray', origin='lower')
#plt.plot()

In [ ]:
#Limit for active pixels in calculation (has to be over 10^6 to work properly, possibly more)
sep.set_extract_pixstack(1000000)
objects = sep.extract(data_sub, 5, err=np.sqrt(1/wht_data),segmentation_map=True)
#change second parameter in order to act on the detection threshold
#optional kernel

In [ ]:
objectlist = []
segmentation = []
objectlist = np.array(objectlist)
segmentation = np.array(segmentation)
objectlist = objects[0]
segmentation = objects[1]

# how many objects were detected + segmentation map (reversed)
print("Number of objects:", len(objectlist))
plt.imshow(segmentation)

In [ ]:
from matplotlib.patches import Ellipse

# plot background-subtracted image
fig, ax = plt.subplots()
m, s = np.mean(data_sub), np.std(data_sub)
im = ax.imshow(data_sub, interpolation='nearest', cmap='gray',
               vmin=m-s, vmax=m+s, origin='lower')

# plot an ellipse for each object
for i in range(len(objectlist)):
    e = Ellipse(xy=(objectlist['x'][i], objectlist['y'][i]),
                width=6*objectlist['a'][i],
                height=6*objectlist['b'][i],
                angle=objectlist['theta'][i] * 180. / np.pi)
    e.set_facecolor('none')
    e.set_edgecolor('red')
    ax.add_artist(e)

In [ ]:
flux, fluxerr, flag = sep.sum_circle(data_sub, objectlist['x'], objectlist['y'],
                                     0.35, err=np.sqrt(1/wht_data), gain=1.0)

# show the first 10 objects results:
for i in range(10):
    print("object {:d}: flux = {:f} +/- {:f}".format(i, flux[i], fluxerr[i]))